In [ ]:
#i "nuget:C:\Users\dicolomb\OneDrive - Microsoft\LocalNugetDev"

In [ ]:
#r "nuget:piTop"
#r "nuget:piTop.FoundationKit"

In [ ]:
using PiTopMakerArchitecture.Foundation;
using PiTopMakerArchitecture.Foundation.Components;
using PiTopMakerArchitecture.Foundation.Sensors;

In [ ]:
var plate = new Plate();

In [ ]:
display(plate, JsonFormatter.MimeType);